In [7]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

from sklearn.feature_selection import VarianceThreshold, RFECV
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC, OneClassSVM
from sklearn.ensemble import RandomForestClassifier, IsolationForest, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from imblearn.over_sampling import RandomOverSampler, SMOTE, KMeansSMOTE, ADASYN, BorderlineSMOTE, \
                                    SMOTENC, SMOTEN, SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler, CondensedNearestNeighbour, TomekLinks, \
                                    EditedNearestNeighbours, NeighbourhoodCleaningRule, OneSidedSelection
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline

import xgboost as xgb



In [8]:
standardscaler= StandardScaler()
normalizer=Normalizer()

preprocessing_1 = [standardscaler, normalizer]

In [10]:
oversamplings = [RandomOverSampler(), SMOTE(), KMeansSMOTE(), ADASYN(), BorderlineSMOTE(),
                 SMOTENC(categorical_features=2), SMOTEN(), SVMSMOTE()]

undersamplings = [RandomUnderSampler(), CondensedNearestNeighbour(), TomekLinks(),
                  EditedNearestNeighbours(), NeighbourhoodCleaningRule(), OneSidedSelection()]

combosampling = [SMOTEENN(), SMOTETomek()]

In [6]:
rfecv = RFECV(estimator=LogisticRegression(class_weight="balanced"), step=1, scoring="neg_mean_squared_error")
pca = PCA()
kpca=KernelPCA()

decompositions = [rfecv, pca, kpca]

In [ ]:
models = [LogisticRegression(class_weight="balanced"), LinearDiscriminantAnalysis ]

In [ ]:
pipe = Pipeline(steps=[('pre1', StandardScaler()),('pre2': Normalizer()), ('classifier', SVC())])

# Add a dict of classifier and classifier related parameters in this list
params_grid = [{
                'classifier':[SVC()],
                'classifier__C': [1, 10, 100, 1000],
                'classifier__gamma': [0.001, 0.0001],
                },
                {
                'classifier': [DecisionTreeClassifier()],
                'classifier__max_depth': [1,2,3,4,5],
                'classifier__max_features': [None, "auto", "sqrt", "log2"],
                },
               # {'classifier':[Any_other_classifier_you_want],
               #  'classifier__valid_param_of_your_classifier':[valid_values]

              ]

grid = GridSearchCV(pipe, params_grid)